In [1]:
import sys
print(sys.version)


3.11.9 (main, Apr  6 2024, 17:59:24) [GCC 11.4.0]


In [2]:
# Define the packages to include
packages = [
    "org.apache.hadoop:hadoop-aws:3.3.4",  # Hadoop-AWS package,
]

In [3]:
# MinIO credentials
minio_endpoint = "http://minio:9000"
minio_access_key = "ouxhmt4Ez73Rt8HSNQFu"
minio_secret_key = "nkEwOH7TsENA7hRSkH0VSRMqdNq65ystE2BODdHW"

In [4]:
from pyspark import SparkConf
conf = (
            SparkConf()
            .setAppName("minIOTest")
            .set("spark.jars.packages", ",".join(packages))
            .set("spark.hadoop.fs.s3a.access.key", minio_access_key)
            .set("spark.hadoop.fs.s3a.secret.key", minio_secret_key)
            .set("spark.hadoop.fs.s3a.endpoint", minio_endpoint)
            .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "true")
            .set("spark.hadoop.fs.s3a.path.style.access", "true")
            .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
            .set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
            .set("spark.hadoop.fs.s3a.attempts.maximum", "1")
            .set("spark.hadoop.fs.s3a.connection.establish.timeout", "5000")
            .set("spark.hadoop.fs.s3a.connection.timeout", "10000")
        )

In [7]:
from pyspark.sql import SparkSession
# Initialize SparkSession

spark =SparkSession.builder.config(conf=conf).master("spark://spark-master:7077").getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.11/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /config/.ivy2/cache
The jars for the packages stored in: /config/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-28a3b313-02d4-4cbf-9136-df40a53d3082;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
downloading https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.262/aws-java-sdk-bundle-1.12.262.jar ...
	[SUCCESSFUL ] com.amazonaws#aws-java-sdk-bundle;1.12.262!aws-java-sdk-bundle.jar (168294ms)
downloading https://repo1.maven.org/maven2/org/wildfly/openssl/wildfly-openssl/1.0.7.Final/wildfly-openssl-1.0.7.Final.jar ...
	[SUCCESSFUL ] org.wildfly.openssl#wildfly-openssl;1.0.7.Final!wildfly-openssl.jar (668ms)
:: resolution report :: resolve 144ms :: artifacts dl 168970ms
	:: modules in use:
	com.amazonaws#aws-java-s

In [8]:
df = spark.read.option("header", "true") \
    .csv(f"s3a://test2/DimCountry.csv")
# show data
df.show()

24/07/16 04:17:15 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+---------+--------------+
|countryid|       country|
+---------+--------------+
|        1|     Argentina|
|        2|     Australia|
|        3|       Austria|
|        4|    Azerbaijan|
|        5|       Belgium|
|        6|        Brazil|
|        7|      Bulgaria|
|        8|        Canada|
|        9|        Cyprus|
|       10|Czech Republic|
|       11|       Denmark|
|       12|         Egypt|
|       13|       Estonia|
|       14|       Finland|
|       15|        France|
|       16|       Germany|
|       17|        Greece|
|       18|       Hungary|
|       19|         India|
|       20|     Indonesia|
+---------+--------------+
only showing top 20 rows



In [16]:
# Write DataFrame to MinIO
df.write.mode("overwrite").csv("s3a://test2/DimCountry2.csv")

24/07/16 04:25:43 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [17]:
spark.stop()